# Index‑Options Risk Snapshot with Polygon.io

This notebook shows how to pull live **Greeks, implied volatility (IV), open‑interest, and price data** for **index options** (e.g., SPX, NDX, RUT, VIX) from the Polygon.io REST API and aggregate basic portfolio‑level risk measures.

**What you’ll learn**

1. How to query the **Option‑Chain Snapshot** endpoint for index underlyings (prefix `I:` per Polygon’s convention).  
2. How to normalise the JSON into a Pandas DataFrame.  
3. How to compute quick aggregate exposures—Δ, Γ, Θ, and ν (vega)—for an illustrative portfolio (here we assume **one contract per option**; multiply by your own position sizes).  

> **Prerequisites**  
> * A live Polygon.io key (`POLYGON_API_KEY` environment variable or paste directly).  
> * `pandas`, `requests`, and (optionally) `pyarrow` for feather/Parquet export.

> **Endpoint used**: `/v3/snapshot/options/{underlying}` (returns price, Greeks, IV, OI, quotes & trades) :contentReference[oaicite:0]{index=0}


In [68]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import requests

# ─── CONFIG ────────────────────────────────────────────────────────────────────
API_KEY    = "6f7zlNdhVy_4zN7ZZHSBLCwHv4cD6Ftw"
END_DATE   = datetime(2025, 7, 18)
OUTPUT_CSV = "SPX_options_reference_chain_20150101_20250718.csv"

# ─── HELPER (same as before) ───────────────────────────────────────────────────
def fetch_contracts_as_of(date_str: str, api_key: str) -> list:
    url = "https://api.polygon.io/v3/reference/options/contracts"
    params = {
        "underlying_ticker": "SPX",
        "as_of": date_str,
        "limit": 1000,
        "apiKey": api_key
    }
    all_results = []
    while url:
        resp = requests.get(url, params=params).json()
        all_results.extend(resp.get("results", []))
        url = resp.get("next_url")
        params = None
    return all_results

# ─── 1) LOAD EXISTING CSV & FIND RESUME DATE ───────────────────────────────────
df_existing = pd.read_csv(OUTPUT_CSV, parse_dates=['snapshot_date'])
last_date = df_existing['snapshot_date'].max()
start_date = last_date + timedelta(days=1)
print(f"Resuming from {start_date.date()} through {END_DATE.date()}")

# ─── 2) LOOP & APPEND NEW DAYS ────────────────────────────────────────────────
new_rows = []
current = start_date
pbar = tqdm(total=(END_DATE - start_date).days + 1, desc="Fetching additional days")
while current <= END_DATE:
    date_str = current.strftime("%Y-%m-%d")
    try:
        contracts = fetch_contracts_as_of(date_str, API_KEY)
    except Exception as e:
        print(f"⚠️ {date_str}: {e}")
        current += timedelta(days=1)
        pbar.update(1)
        continue

    for c in contracts:
        new_rows.append({
            "snapshot_date":    date_str,
            "contract_symbol":  c.get("ticker"),
            "underlying_ticker":c.get("underlying_ticker"),
            "expiration_date":  c.get("expiration_date"),
            "strike_price":     c.get("strike_price"),
            "option_type":      c.get("contract_type"),
            # …and any other fields you already included…
        })

    current += timedelta(days=1)
    pbar.update(1)
pbar.close()

# ─── 3) CONCAT & WRITE BACK ────────────────────────────────────────────────────
if new_rows:
    df_new = pd.DataFrame(new_rows)
    df_full = pd.concat([df_existing, df_new], ignore_index=True)
    df_full.to_csv(OUTPUT_CSV, index=False)
    print(f"Appended {len(df_new)} new records; CSV now has {len(df_full)} rows.")
else:
    print("Nothing new to append—you're fully up to date!")


Resuming from 2025-02-23 through 2025-07-18


Fetching additional days: 100%|██████████████████████████████████████████████████████| 146/146 [00:49<00:00,  2.94it/s]


Appended 10000 new records; CSV now has 209000 rows.


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
from requests.adapters import HTTPAdapter, Retry

# ─── CONFIG ───────────────────────────────────────────────────────────────
API_KEY   = "6f7zlNdhVy_4zN7ZZHSBLCwHv4cD6Ftw"
INPUT_CSV = "SPX_options_reference_chain_20150101_20250718.csv"
OUTPUT_CSV= "SPX_options_reference_chain_20150101_20250718_populated.csv"

# ─── 1) LOAD CSV ──────────────────────────────────────────────────────────
df = pd.read_csv(
    INPUT_CSV,
    parse_dates=['snapshot_date', 'expiration_date'],
    dtype={'contract_symbol': str}
)

# ─── 2) SET UP A SESSION WITH RETRIES ─────────────────────────────────────
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.5,
                status_forcelist=[429, 500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

# ─── 3) PREPARE METADATA DICT ─────────────────────────────────────────────
# Ensure these columns exist (so merge will fill them)
for col in ['exercise_style','multiplier','expiration_type',
            'listing_date','updated_at','active']:
    df[col] = pd.NA

symbols = df['contract_symbol'].dropna().unique()
metadata = {}

# ─── 4) FETCH PER SYMBOL WITH ERROR HANDLING ──────────────────────────────
for sym in tqdm(symbols, desc="Fetching metadata"):
    url = f"https://api.polygon.io/v3/reference/options/contracts/{sym}"
    try:
        resp = session.get(url, params={"apiKey": API_KEY}, timeout=10).json()
        c = resp.get("results", {}) or {}
        metadata[sym] = {
            'exercise_style':   c.get('exercise_style'),
            'multiplier':       c.get('multiplier'),
            'expiration_type':  c.get('expiration_type'),
            'listing_date':     c.get('listing_date'),
            'updated_at':       c.get('updated_at'),
            'active':           c.get('active')
        }
    except Exception as e:
        # On error, fill with NaNs but continue
        metadata[sym] = dict.fromkeys(
            ['exercise_style','multiplier','expiration_type',
             'listing_date','updated_at','active'],
            pd.NA
        )

# ─── 5) MERGE METADATA BACK ────────────────────────────────────────────────
meta_df = pd.DataFrame.from_dict(
    metadata, orient='index'
).rename_axis('contract_symbol').reset_index()

df = df.drop(
    columns=['exercise_style','multiplier','expiration_type',
             'listing_date','updated_at','active'],
    errors='ignore'
).merge(meta_df, on='contract_symbol', how='left')

# ─── 6) SAVE FINAL CSV ─────────────────────────────────────────────────────
df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Populated metadata for all symbols and saved to {OUTPUT_CSV}")


Fetching metadata:   0%|                                                         | 18/35902 [01:18<37:53:13,  3.80s/it]